In [ ]:
import robin_stocks as rs #the robinhood api wrapper
import pandas as pd #for creating CSV files
from datetime import timedelta, datetime #for getting current time and converting recent news timestamps
import time #for sleep timer
import csv #for creating CSV file of stock purchase list

In [ ]:
#include functions from notebooks
%run auth.ipynb
%run recent_stock_news.ipynb
%run check_volume.ipynb
%run get_trending_stocks.ipynb
%run purchase_stock.ipynb
%run tradeable_stocks.ipynb
%run cloud_CSV_storage.ipynb

In [ ]:
#authenticate robinhood account
get_header = rh_auth(username='', password='')

In [ ]:
def market_is_open():
    #get current time
    current_time = datetime.now()
    #format currrent time
    current_time_form = (current_time.hour * 100) + current_time.minute
    if current_time_form >= 600 and current_time_form <= 1230:
        return True
    else:
        return False

In [ ]:
def trade():
    #get current time
    current_time = datetime.now()
    #Inform user that process is beginning again
    print("Algorithm starting at ", current_time.hour, current_time.minute)
    
    #apply filters to tradeable stocks
    #push_stock_purchase_list_to_cloud(all_tradeable_stocks)
    filter_tradeable_stocks(all_tradeable_stocks)
    
    #get purchase list 
    #purchase_list = get_purchase_list_from_cloud()
    purchase_list = get_sorted_purchase_list()
    
    #if there is more than 0 stocks in purchase list
    if len(purchase_list) > 0:
        #send purchase list to buy stock which tries to purchase stocks
        buy_stock(purchase_list)

In [ ]:
all_tradeable_stocks = get_tradeable_stocks()

In [ ]:
while True:
    if market_is_open():
        trade()